<a href="https://colab.research.google.com/github/Elbereth-Elentari/Book_recommender/blob/master/Catalogue_annotation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install spacy==2.3.0
!pip3 install spacy_langdetect
!pip3 install selenium
!apt-get install chromium-chromedriver
!pip3 install tqdm
import spacy.cli
spacy.cli.download('en_core_web_lg')
spacy.cli.download('pl_core_news_lg')

In [6]:
import os
import string
import re
import pandas as pd

import spacy
import en_core_web_lg
nlp = en_core_web_lg.load()
import pl_core_news_lg
nlp_pl = pl_core_news_lg.load()

from tqdm.notebook import tqdm

In [33]:
def calculate_similarity(preprocessed_df, tag_df):
    cat_sim = pd.DataFrame()
    for lang in ['en', 'pl']:
        df = preprocessed_df[preprocessed_df['language'] == lang]
        semantics = ' '.join(tag_df[tag_df['language'] == lang].tokens.str.join(' '))
        if lang == 'en': model = nlp
        elif lang == 'pl': model = nlp_pl
        semantics = model(semantics)

        similarity = []
        for doc in tqdm(model.pipe(df['title'].values, batch_size=200), desc=f'Calculating semantic similarity for {lang}', total=len(df)):
            if doc.is_parsed: similarity.append(semantics.similarity(doc))
            else: preprocessed.append('similarity_fail')
        df['similarity'] = similarity
        cat_sim = cat_sim.append(df)
    cat_sim.drop(cat_sim[cat_sim['similarity'] < 0.5].index, inplace=True)
    cat_sim.sort_values(by='similarity', inplace=True, ascending=False)
    cat_sim.drop(columns={'tokens'}, inplace=True)
    return cat_sim

In [4]:
cat = pd.read_json('/content/drive/My Drive/Library_catalogue_preprocessed.jl', lines=True)

In [34]:
cat_sim = calculate_similarity(cat[cat['interesting'] == ''], cat[cat['interesting'] == 'yes'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  if sys.path[0] == '':


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [46]:
def annotate_catalogue(cat, mode=''):
    while True:
        if not mode:
            print('Enter mode: title startswith, title contains, author startswith')
            mode = input()
        if mode == 'title startswith':
            print('Enter the beginning of the title or change mode.')
            query = input()
            cat_part = cat[cat['title'].str.lower().str.startswith(query.lower())]
        elif mode == 'title contains':
            print('Enter a word from the title or change mode.')
            query = input()
            cat_part = cat[cat['title'].str.lower().str.contains(query.lower())]
        elif mode == 'author startswith':
            print('Enter the author or change mode.')
            query = input()
            cat_part = cat[cat['author'].str.lower().str.startswith(query.lower())]
        else:
            break

        if query == '': break
        elif query == 'change mode':
            mode = ''
            continue

        cat = assess_save(cat_part, cat)
    return cat

In [29]:
def assess_save(partial_catalogue, cat):
    for index, row in partial_catalogue.iterrows():
        print(row.year, row.author, row.title)
        print('Is this book interesting? y, n or enter')
        assessment = input()
        if assessment == 'y': cat.at[index, 'interesting'] = 'yes'
        elif assessment == 'n': cat.at[index, 'interesting'] = 'no'
    cat.to_json('/content/drive/My Drive/Library_catalogue_preprocessed.jl', lines=True, orient='records')
    return cat

In [ ]:
for index, row in cat_sim[(cat_sim['language'] == 'pl') & (cat_sim['year'] > 1999)][50:100].iterrows():
    cat_part = cat[cat['title'].str.lower() == row.title.lower()]
    cat = assess_save(cat_part, cat)

In [ ]:
cat = annotate_catalogue(cat)

In [65]:
len(cat[cat['interesting'] == 'yes'])

477

In [ ]:
cat.to_json('/content/drive/My Drive/Library_catalogue_preprocessed.jl', lines=True, orient='records')